In [1]:
import nltk
import numpy as np
import json

## Load Corpus

In [2]:
# Fungsi untuk load data corpus dari file txt
# Return dari fungsi ini adalah string yang berisi corpus topik tertentu
def loadCorpus(file_name):
    p = ''
    with open(file_name, 'r') as corpus:
        for data in corpus:
            sentences = data.split('\t')
            p += sentences[0].replace('\n', ' ')
    
    return p.lower()

In [3]:
# Panggil fungsi corpus dan simpan hasilnya ke variable corpus
corpus = loadCorpus('plain/minicorpus.txt')
# Melakukan tokenisasi corpus
token = nltk.word_tokenize(corpus)

In [4]:
print(corpus)

saya suka makan nasi. saya suka kamu, kamu suka dia. tidur di kasur. kasur tempat untuk tidur. makanan yang saya suka adalah nasi padang. saya makan nasi di warung. warung nasi padang adalah tempat favorit saya. makanan di warung nasi padang yang sangat saya suka yaitu nasi rendang. rasanya sangat enak. makanan kesukaan saya selain rendang adalah ayam goreng. saya pergi ke toko buku untuk membeli sebuah buku. buku yang saya beli berwarna biru. buku biru itu adalah milik saya.


In [5]:
print(token)

['saya', 'suka', 'makan', 'nasi', '.', 'saya', 'suka', 'kamu', ',', 'kamu', 'suka', 'dia', '.', 'tidur', 'di', 'kasur', '.', 'kasur', 'tempat', 'untuk', 'tidur', '.', 'makanan', 'yang', 'saya', 'suka', 'adalah', 'nasi', 'padang', '.', 'saya', 'makan', 'nasi', 'di', 'warung', '.', 'warung', 'nasi', 'padang', 'adalah', 'tempat', 'favorit', 'saya', '.', 'makanan', 'di', 'warung', 'nasi', 'padang', 'yang', 'sangat', 'saya', 'suka', 'yaitu', 'nasi', 'rendang', '.', 'rasanya', 'sangat', 'enak', '.', 'makanan', 'kesukaan', 'saya', 'selain', 'rendang', 'adalah', 'ayam', 'goreng', '.', 'saya', 'pergi', 'ke', 'toko', 'buku', 'untuk', 'membeli', 'sebuah', 'buku', '.', 'buku', 'yang', 'saya', 'beli', 'berwarna', 'biru', '.', 'buku', 'biru', 'itu', 'adalah', 'milik', 'saya', '.']


## Modeling

In [6]:
# Fungsi untuk membuat model unigram
# Return dari fungsi ini yaitu dictionary yang berisi unigram dari kata yang ada dalam corpus
def unigram(corpus):
    a = {}
    for word in token:
        if word not in a:
            a[word] = 1
        else:
            a[word] += 1
    return a

In [7]:
# Buat unigram dari corpus yang sudah di load kemudian disimpan kedalam variable ugram
ugram = unigram(corpus)

In [8]:
print(ugram)
len(ugram)

{'saya': 10, 'suka': 5, 'makan': 2, 'nasi': 6, '.': 13, 'kamu': 2, ',': 1, 'dia': 1, 'tidur': 2, 'di': 3, 'kasur': 2, 'tempat': 2, 'untuk': 2, 'makanan': 3, 'yang': 3, 'adalah': 4, 'padang': 3, 'warung': 3, 'favorit': 1, 'sangat': 2, 'yaitu': 1, 'rendang': 2, 'rasanya': 1, 'enak': 1, 'kesukaan': 1, 'selain': 1, 'ayam': 1, 'goreng': 1, 'pergi': 1, 'ke': 1, 'toko': 1, 'buku': 4, 'membeli': 1, 'sebuah': 1, 'beli': 1, 'berwarna': 1, 'biru': 2, 'itu': 1, 'milik': 1}


39

In [9]:
# Melakukan unigram terhadap token yang sudah diinisialisasi
# Tujuannya untuk mendapatkan kata-kata yang unik dari corpus yang ada
token_unigram = []

for word in token:
    if (word not in token_unigram):
        token_unigram.append(word)

In [10]:
len(token_unigram)

39

In [11]:
# Inisialisasi bgram dengan jumlah setiap pasangan kata adalah 0
dict_bgram = {}
for word_x in token_unigram:
    a = {}
    for word_y in token_unigram:
        a[word_y] = 0
        
    dict_bgram[word_x] = a

In [12]:
# Update table bgram jika menemukan pasangan kata yang sesuai di dalam corpus
# Jika pasangan kata ditemukan maka jumlah pasangan kata tersebut bertambah
token = np.array(token)
for word in token_unigram:
    # Mencari index dari kata pertama
    index = np.where(token == word)
    a = []
    
    # Mencari kata selanjutnya dari kata sebelumnya (word[i+1])
    # Kemudian simpan ke dalam array
    for i in index[0]:
        if (i+1 != len(token)):
            a.append(token[i+1])
    
    # Menjumlahkan kata yang sudah ada di table bgram jika menemukan pasangan kata yang serupa
    for word2 in a:
        dict_bgram[word][word2] += 1

In [13]:
# Menghitung peluang table bgram
def calculate_prob_bgram(dict_bgram, ugram):
    a = {}
    for word1 in dict_bgram:
        b = {}
        for word2 in dict_bgram[word1]:
            b[word2] = dict_bgram[word1][word2]/ugram[word1]
        a[word1] = b
    return a

In [14]:
dict_prob_bgram = calculate_prob_bgram(dict_bgram, ugram)

In [15]:
# Simpan file bgram dalam format json
with open('bgram.json', 'w') as f:
    json.dump(dict_prob_bgram, f)

## Predict Input Sentence

In [16]:
# Menghitung probabilitas dari kalimat berdasarkan model bgram
def calculate_prob_input(dict_prob_bgram, kalimat_input):
    hasil = 1
    for i in range(len(kalimat_input)-1):
        word = kalimat_input
        if (word[i] in dict_prob_bgram.keys() and word[i+1] in dict_prob_bgram.keys()):
            hasil *= dict_prob_bgram[word[i]][word[i+1]]
        else: 
            hasil *= 0
            break
    return hasil

In [17]:
# Memprediksi kata selanjutnya yang mungkin dan memiliki probabilitas yang besar
def predict_best_prob(dict_prob_bgram, kalimat_input):
    newKalimat = kalimat_input
    for i in range(len(kalimat_input)-1):
        currWord = kalimat_input[i]
        nextWord = kalimat_input[i+1]
        if (currWord in dict_prob_bgram.keys() and nextWord in dict_prob_bgram.keys()):
            a = [k for k, v in dict_prob_bgram[currWord].items() if v == max(dict_prob_bgram[currWord].values())]
            if (nextWord not in a):
                nextWord = a[0]
            newKalimat[i+1] = nextWord
        else:
            print('Kata ', currWord, ' tidak ada di dalam corpus')
            
    return newKalimat

In [18]:
# Fungsi untuk mengambil data kamus
def loadKamus():
    a = []
    with open('english_indonesia.txt', 'r') as kamus:
        for data in kamus:
            k = data.split('\t')
            token_arti = nltk.word_tokenize(k[1])
            a.append([k[0], token_arti])
    return a

In [19]:
# Fungsi untuk membersihkan kamus dari karakter yang tidak diperlukan
# Fungsi ini mengembalikan 1 kata pertama terjemahan bahasa inggris - indonesia
def clearKamus(kamus):
    a = {}
    b = ['ks', 'kt', 'kb', 'kkt', 'ks', 'kki', 'kk', 'kb..', 'ks.', 'kg', '.', '(', ')', 
         '1', '2', '3', '4', '5', '.1', 'kb1.', '[', ']', ',']
    for data in kamus:
        bb = [elm for elm in data[1] if elm not in b]
        a[data[0]] = bb[0]
        
    return a

In [20]:
kamus = loadKamus()
clear_kamus = clearKamus(kamus)

In [21]:
def execute(kalimat_input):    
    kalimat_input = kalimat_input.lower()
    token_kalimat = nltk.word_tokenize(kalimat_input)

    terjemahan = []

    for kata in token_kalimat:
        terjemahan.append(clear_kamus[kata])

    print('Kalimat input: ', kalimat_input)
    print()
    print('Terjemahan: ', terjemahan)
    prob = calculate_prob_input(dict_prob_bgram, terjemahan)
    print('Probabilitas: ', prob)

    newKalimat = predict_best_prob(dict_prob_bgram, terjemahan)
    print()
    newProb = calculate_prob_input(dict_prob_bgram, newKalimat)
    print('Best Predict: ', newKalimat)
    print('Probabilitas: ', newProb)

In [22]:
execute('i like you')

Kalimat input:  i like you

Terjemahan:  ['saya', 'kesukaan', 'kamu']
Probabilitas:  0.0

Best Predict:  ['saya', 'suka', 'kamu']
Probabilitas:  0.08000000000000002


In [23]:
execute('i eat rice')

Kalimat input:  i eat rice

Terjemahan:  ['saya', 'eats', 'padi']
Probabilitas:  0
Kata  saya  tidak ada di dalam corpus
Kata  eats  tidak ada di dalam corpus

Best Predict:  ['saya', 'eats', 'padi']
Probabilitas:  0


In [24]:
execute('i eat on the bed')

Kalimat input:  i eat on the bed

Terjemahan:  ['saya', 'eats', 'sedang', 'kst', 'tempat']
Probabilitas:  0
Kata  saya  tidak ada di dalam corpus
Kata  eats  tidak ada di dalam corpus
Kata  sedang  tidak ada di dalam corpus
Kata  kst  tidak ada di dalam corpus

Best Predict:  ['saya', 'eats', 'sedang', 'kst', 'tempat']
Probabilitas:  0


In [25]:
execute('i sleep in a stall')

Kalimat input:  i sleep in a stall

Terjemahan:  ['saya', 'tidur', 'Inf', 'huruf', 'kedai']
Probabilitas:  0.0
Kata  suka  tidak ada di dalam corpus
Kata  Inf  tidak ada di dalam corpus
Kata  huruf  tidak ada di dalam corpus

Best Predict:  ['saya', 'suka', 'Inf', 'huruf', 'kedai']
Probabilitas:  0.0


In [26]:
execute('i bought a blue book')

Kalimat input:  i bought a blue book

Terjemahan:  ['saya', 'lih', 'huruf', 'biru', 'buku']
Probabilitas:  0
Kata  saya  tidak ada di dalam corpus
Kata  lih  tidak ada di dalam corpus
Kata  huruf  tidak ada di dalam corpus

Best Predict:  ['saya', 'lih', 'huruf', 'biru', '.']
Probabilitas:  0
